### LIBRARY AND FONCTIONS

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import requests
import multiprocessing
import pandas as pd

from tqdm.notebook import tqdm
from tqdm import tqdm
from astropy.wcs import WCS
from astropy.io import fits
from apertphoto_modified import AstroTools
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
def get_one_sciimg(year, month, day, fractime, field, filter_band, ccd, quadrant, output_dir,suffixe):
    imgtypecode = 'o'
    zeros = '00'
    datefull = f"{year}{month}{day}{fractime}"
    prefixe = f"ztf_{datefull}_{zeros}{field}_{filter_band}_c{ccd}_{imgtypecode}_q"
    nom = f"{prefixe}{quadrant}{suffixe}"

    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, nom)

    base_url = 'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/'
    timed_url = f"{base_url}/{year}/{month}{day}/"
    fractimed_url = f"{timed_url}/{fractime}/"
    url = f"{fractimed_url}{nom}"
    #print(url)

    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    if response.status_code == 200:
        with open(filepath, 'wb') as f:
            for data in tqdm(response.iter_content(1024), total=total_size // 1024, unit='KB'):
                f.write(data)

def download_images_from_csv(csv_path, target_field, filter_band, ccd, quadrant, start_date, end_date,suffixe):

    output_dir = f"ztf_data/00{target_field}_{filter_band}_c{ccd}_o_q{quadrant}"
    os.makedirs(output_dir, exist_ok=True)

    with open(csv_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        if target_field not in reader.fieldnames:
            #print(f"Field '{target_field}' not found in CSV columns.")
            return

        for row in reader:
            filename = row[target_field]
            if filename:
                date_part = filename[0:8]
                if start_date <= date_part <= end_date:
                    year = filename[0:4]
                    month = filename[4:6]
                    day = filename[6:8]
                    fractime = filename[8:]
                    get_one_sciimg(year, month, day, fractime, target_field, filter_band, ccd, quadrant, output_dir,suffixe)


def telecharge_un_quadrant(args,Field,Filter,START,END):
    ccd, quad = args
    ccd_str = f"{ccd:02d}"
    quad_str = str(quad)
    try:
        download_images_from_csv(
            csv_path="files_by_field.csv",
            target_field=Field,
            filter_band=Filter,
            ccd=ccd,
            quadrant=quad,
            start_date=START,
            end_date=END,
            suffixe = '_sciimg.fits'        #'_msking.fits' or '_sciimg.fits'       
            )       
        return f"✅ CCD {ccd_str} Q{quad_str}"
    except Exception as e:
        return f"❌ CCD {ccd_str} Q{quad_str} : {e}"

        
def pixel_to_radec(x, y, wcs):
    skycoord = wcs.pixel_to_world(x, y)
    ra = skycoord.ra.deg  
    dec = skycoord.dec.deg 
    return ra, dec

### MAIN SETTINGS

In this section, you'll choose which data you're going to use.

Select the area of sky you wish to observe, then the time range and color filter.

Your sky area will be divided into 16 CCDs of 4 quadrants each.

In [ ]:
# SETTINGS

Field = "0664"          #from 0001 to 1800 approximately, see *ZTF Explanatory Supplement* to futher more informations.
Filter = "zr"           #zr, zg or zi
Ccd = "01"              #01 to 16
Quadrant = "1"          #1 to 4
START = "20200101"      #not before 20180321
END = "20200501"        #not after 20210108

FFCQ = f'00{Field}_{Filter}_c{Ccd}_o_q{Quadrant}'
fits_files = sorted(glob.glob(f".\\ztf_data\\{FFCQ}\\*.fits"))
aligned_files = sorted(glob.glob(f".\\ztf_data_aligned\\{FFCQ}\\*.fits"))
ref_filename = f'.\\ztf_data_ref\\{FFCQ}\\refimg_median_{FFCQ}.fits'

### DOWNLOAD

For the Download section, you have two ways to get your data :
- You can just download data from only one readout-channel (quadrant)
- You can download several quadrants at the same time.

Be careful, for the full data in red (with one quadrant), you need up to **_30 GB on your disk space_** for the full process.

In [ ]:
# ONE DOWNLOAD

download_images_from_csv(
    csv_path="fichiers_par_field.csv",
    target_field=Field,             
    filter_band=Filter,
    ccd=Ccd,
    quadrant=Quadrant,
    start_date=START,
    end_date=END,
    suffixe = '_sciimg.fits'        #'_msking.fits' or '_sciimg.fits'
)

In [ ]:
# MULTIPLES DOWNLOADS

# Crée la liste des tâches de quadrant (classique)
args_list = [(ccd, quad) for ccd in range(1, 5) for quad in range(1, 5)]

# CPU parallel multi-tasking (on process)
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(telecharge_un_quadrant, args,Field,Filter,START,END) for args in args_list]
    for future in tqdm(as_completed(futures), total=len(futures)):
        print(future.result())


In [ ]:
# SEEING STATS


with fits.open(fits_files[0]) as hdu_list:
        hdu0 = hdu_list[0].header
        wcs_0 = WCS(hdu0)
ra , dec = pixel_to_radec(1400, 600, wcs_0)
print(f"RA: {ra}, DEC: {dec}")

n = 100
basename_median = os.path.basename(f'.\\ztf_data_ref\\{FFCQ}\\refimg_median_{FFCQ}.fits')
basename_mean = os.path.basename(f'.\\ztf_data_ref\\{FFCQ}\\refimg_mean_{FFCQ}.fits')
AstroTool = AstroTools(fits_files, file_name=FFCQ)
# Chargement des données FITS
"""img1 = fits.open(f'.\\ztf_data_ref\\{FFCQ}\\refimg_median_{FFCQ}.fits')[0].data
img2 = fits.open(f'.\\ztf_data_ref\\{FFCQ}\\refimg_mean_{FFCQ}.fits')[0].data
AstroTool.show_img(img1, title=f'{basename_median}', vmin=np.nanmedian(img1), vmax=np.nanmedian(img1) +2)
AstroTool.show_img(img2, title=f'{basename_mean}', vmin=np.nanmedian(img1), vmax=np.nanmedian(img1) +2)"""

mean_seeing, median_seeing , sigma_seeing , seeing_max = AstroTool.distrib_seeing(fits_file=None , Plotshow = False)

In [ ]:
# ALIGNEMENT

AstroTool.align_images()

In [ ]:
# REFERENCE IMAGE

aligned_files = sorted(glob.glob(f".\\ztf_data_aligned\\{FFCQ}\\*.fits")) # Je le redéfini ici car avant l'alignement il n'y avait pas de fichiers dans ce dossier

AstroTool.get_refimg(aligned_files[:2], ref_method='mean', seeing_max=6, zp_ref=27.5, limit_bkg=300, Imshow_ref=True, Imshow=False, output_file='ztf_data_stack', n='') # ~ 3 minutes (for ~160 images)

In [ ]:
# STACKING IMAGES

# Liste finale contenant les chemins des fichiers validés
valid_fits_files = []

root_dir = f".\\ztf_data_aligned\\{FFCQ}\\"

# Parcourir les sous-dossiers
for subdir, _, files in os.walk(root_dir):
    for filename in files:
        if filename.endswith(".fits"):
            filepath = os.path.join(subdir, filename)
            try:
                with fits.open(filepath) as hdul:
                    header = hdul[0].header
                    seeing = header["SEEING"]
                    data = hdul[0].data
                    
                    # Vérifie que les deux conditions sont remplies
                    if seeing is not None and seeing <= 6:
                        if data is not None:
                            median_value = np.nanmedian(data)
                            if median_value < 400:
                                valid_fits_files.append(filepath)
            except Exception as e:
                print(f"Erreur lors de la lecture de {filepath}: {e}")

print(filepath)

# Afficher ou enregistrer la liste
for path in valid_fits_files:
    print(path)

# Paramètres que tu veux passer à la fonction (ajuste-les si besoin)
ref_method = 'median'
seeing_max = 6
zp_ref = 27.5
limit_bkg = 400
Imshow_ref = False
Imshow = False
output_file_base = 'ztf_data_stack'

# Lancer la fonction toutes les 3 images
for i in range(0, len(valid_fits_files), 3):
    batch = valid_fits_files[i:i+3]
    if len(batch) >= 2:  # On ne lance la fonction que si on a au moins 2 fichiers
        output_file = f'{output_file_base}'
        print(f"Lancement de get_refimg sur le lot {i//3} : {batch}")
        AstroTool.get_refimg(
            batch,  # équivalent de aligned_files[:2]
            ref_method=ref_method,
            seeing_max=seeing_max,
            zp_ref=zp_ref,
            limit_bkg=limit_bkg,
            Imshow_ref=Imshow_ref,
            Imshow=Imshow,
            output_file=output_file,
            n=f'{i}'
        )

In [ ]:
stack_files = sorted(glob.glob(f".\\ztf_data_stack\\{FFCQ}\\*.fits"))

ref_filename = f'.\\ztf_data_ref\\{FFCQ}\\refimg_{FFCQ}.fits'
Start_Year , Start_Month , Start_Day  = 2018, 4, 6
End_Year , End_Month , End_Day  = 2021, 1, 8
final_table = AstroTool.subtraction(stack_files, ref_filename , ra , dec , Start_Year, Start_Month, Start_Day , End_Year, End_Month, End_Day, zp=None, limit_bkg=400, Imshow=True, zoom=50 , coef_img_ref=1)
final_table


# Simulation d'une liste de dates à partir du nom des fichiers si besoin :
# Sinon, adapter en ajoutant un return dates, fracday_list dans subtraction()
# dates = ['2020-01-01', '2020-01-03', '2020-01-07', ...]  # doit correspondre à final_table

# Juste un exemple de chronologie linéaire si t'as pas les dates (sinon récupère-les)
jours = np.arange(len(final_table))

# Tracé de la light curve
plt.figure(figsize=(10,5))
plt.plot(jours, final_table['flux_r'], 'o-', label='Flux')
plt.xlabel("Jour (index)")
plt.ylabel("Flux (ADU)")
plt.title("Courbe de lumière d’un point")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()